# SFT example

based on: https://gist.github.com/lkluft/ddda28208f7658d93f8238ad88bd45f2

In [13]:
# Find all txt-files on remote machine.
# 
# /scratch/lehre/edvprak/u300509/
# ├── bar
# │   ├── bar1
# │   │   ├── test.nc
# │   │   └── test.txt
# │   └── bar2
# │       └── test.txt
# └── foo
#     ├── test2.txt
#     └── test.pdf
# 
# 4 directories, 5 files

import os
import re
import stat
import tempfile

import paramiko


def paramiko_glob(path, pattern, sftp):
    """Search recursively for files matching a given pattern.
    
    Parameters:
        path (str): Path to directory on remote machine.
        pattern (str): Python re [0] pattern for filenames.
        sftp (SFTPClient): paramiko SFTPClient.
        
    [0] https://docs.python.org/2/library/re.html
        
    """
    p = re.compile(pattern)
    root = sftp.listdir(path)
    file_list = []
    
    # Loop over all entries in given path...
    for f in (os.path.join(path, entry) for entry in root):
        f_stat = sftp.stat(f)
        # ... if it is a directory call paramiko_glob recursively.
        if stat.S_ISDIR(f_stat.st_mode):
            file_list += paramiko_glob(f, pattern, sftp)
        # ... if it is a file, check the name pattern and append it to file_list.
        elif p.match(f):
            file_list.append(f)
    return file_list


# Setup paramiko connection.
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('localhost', port=2222, username='foo', password='bar')
sftp = ssh.open_sftp()

# Actucal call of paramiko_glob.
files = paramiko_glob('upload/', '', sftp)
print(files)

# Path to temporary directory in RAM disk.
#tmpdir = '/dev/shm/u300509/'
#for f in files:
#    # Copy remote file to local tmp path and read it.
#    local_tmp = tempfile.mktemp(prefix=tmpdir)
#    sftp.get(f, local_tmp)
#    with open(local_tmp) as ftmp:
#        print(ftmp.readlines())
#    os.remove(local_tmp)
        
# Remove temporary files and close connections.
sftp.close()
ssh.close()

['upload/foo-bar-baz']


In [3]:
# !pip install paramiko